In [4]:
import sqlite3 as sq1
import pandas as pd

In [5]:
conn =sq1.connect("RSV.db")

try:
    conn.execute("""
    DROP TABLE IF EXISTS sequence
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS metadata
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS phylogenetic
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS source
    """)
    
    conn.execute("""
    DROP TABLE IF EXISTS epitope
    """)
    
    conn.execute("""
    CREATE TABLE IF NOT EXISTS sequence (assession TEXT, seq TEXT, PRIMARY KEY ('assession'));
    """)
    conn.execute("""
    CREATE TABLE IF NOT EXISTS metadata (assession TEXT, country TEXT, Year INT,
                    FOREIGN KEY('assession') 
                    REFERENCES sequence ('assession')
                    ON DELETE CASCADE
                    ON UPDATE CASCADE);
    """)
    conn.execute("""
    CREATE TABLE IF NOT EXISTS ref (assession TEXT, title TEXT, journal TEXT,
                    FOREIGN KEY('assession') 
                    REFERENCES sequence ('assession')
                    ON DELETE CASCADE
                    ON UPDATE CASCADE);  
    """)
    
    conn.execute("""
    CREATE TABLE IF NOT EXISTS phylogenetic (assession TEXT, genotype TEXT,
                    FOREIGN KEY('assession') 
                    REFERENCES sequence ('assession')
                    ON DELETE CASCADE
                    ON UPDATE CASCADE);  
    """)
    
    conn.execute("""
    CREATE TABLE IF NOT EXISTS epitope (id TEXT, epitope TEXT,
                    FOREIGN KEY('id') 
                    REFERENCES sequence ('id')
                    ON DELETE CASCADE
                    ON UPDATE CASCADE);  
    """)
    
    conn.executemany("""
    INSERT INTO sequence VALUES (?, ?)""",[('AY911262','ATGGACTCTTACT'),
                                           ('FJ614813','CTGGACGGTTTCC'),
                                           ('FJ948820','GCTTTCGGGATAC'),
                                           ('JF920046','ATGGACTCTTACT'),
                                           ('JF920047','GCTTTCGGGATAC')])
    conn.executemany("""
    INSERT INTO metadata VALUES (?,?,?)""",[('AY911262','USA',2007),
                                           ('FJ614813','USA',2009),
                                           ('FJ948820','Netherland',2010),
                                           ('JF920046','USA',2012),
                                           ('JF920047','USA',2009)])
    conn.executemany("""
    INSERT INTO ref VALUES (?,?,?)""",[('AY911262','Respiratory syncytial virus nonstructural proteins NS1 and NS2','J. Virol'),
                                      ('FJ614813','A chimeric A2 strain of respiratory syncytial virus','J. Virol'),
                                      ('JF920047','Whole Genome Sequencing and Evolutionary','PLoS ONE 6')])
    conn.executemany("""
    INSERT INTO phylogenetic VALUES (?, ?)""",[('FJ614813','ON1'),
                                               ('FJ948820','GA2'),
                                               ('JF920046','BA'),
                                               ('JF920047','GB3')])
    
    conn.executemany("""
    INSERT INTO epitope VALUES (?, ?)""",[('AY911262','ATGGAC'),
                                          ('FJ614813','CTGGAC'),
                                          ('FJ948820','AGATAC'),
                                          ('JF920046','ATGGACT'),
                                          ('JF920047','GCTTTCG')])

    cursor1 = conn.execute("select * from sequence")
    df1 = pd.DataFrame(cursor1.fetchall(),columns = [ 'assession','seq'] )              
    print(df1)
    
    cursor2 = conn.execute('''select sequence.assession, sequence.seq, epitope.epitope from
                           sequence join epitope
                           on sequence.assession = epitope.id''')
                           
    df2 = pd.DataFrame(cursor2.fetchall(),columns = [ 'assession','seq','epitope'] )              
    print(df2)
    
    cursor3 = conn.execute('''select sequence.assession, sequence.seq, metadata.country from
                           sequence join metadata
                           on sequence.assession = metadata.assession
                           where country = 'USA' ''')
    df3 = pd.DataFrame(cursor3.fetchall(),columns = [ 'assession','seq','country'] )              
    print(df3)
    
    cursor4 = conn.execute('''select sequence.assession, sequence.seq, phylogenetic.genotype from 
                           sequence join phylogenetic
                           on sequence.assession = phylogenetic.assession
                           order by sequence.assession asc''')
    df4 = pd.DataFrame(cursor4.fetchall(),columns = [ 'assession','seq','genotype'] )              
    print(df4)
    
    cursor5 = conn.execute('''update phylogenetic set genotype = 'GA2' where genotype = 'ON1';''')
    cursor5 = conn.execute('''select metadata.assession, metadata.country, phylogenetic.genotype from
                           metadata join phylogenetic
                           on metadata.assession = phylogenetic.assession''')
    df5 = pd.DataFrame(cursor5.fetchall(),columns = [ 'assession','country','genotype'] )              
    print(df5)                      
                           
                           
    conn.commit();
                           
                           
except sqlite3.Error as e:
    print("There was error:"+str(e))   
    
finally: 
    print("this excuted")
    conn.close()                  
 

  assession            seq
0  AY911262  ATGGACTCTTACT
1  FJ614813  CTGGACGGTTTCC
2  FJ948820  GCTTTCGGGATAC
3  JF920046  ATGGACTCTTACT
4  JF920047  GCTTTCGGGATAC
  assession            seq  epitope
0  AY911262  ATGGACTCTTACT   ATGGAC
1  FJ614813  CTGGACGGTTTCC   CTGGAC
2  FJ948820  GCTTTCGGGATAC   AGATAC
3  JF920046  ATGGACTCTTACT  ATGGACT
4  JF920047  GCTTTCGGGATAC  GCTTTCG
  assession            seq country
0  AY911262  ATGGACTCTTACT     USA
1  FJ614813  CTGGACGGTTTCC     USA
2  JF920046  ATGGACTCTTACT     USA
3  JF920047  GCTTTCGGGATAC     USA
  assession            seq genotype
0  FJ614813  CTGGACGGTTTCC      ON1
1  FJ948820  GCTTTCGGGATAC      GA2
2  JF920046  ATGGACTCTTACT       BA
3  JF920047  GCTTTCGGGATAC      GB3
this excuted


NameError: name 'sqlite3' is not defined